In [1]:
'''
Utils
'''
from collections import defaultdict as dd
from termcolor import colored, cprint
import numpy as np

## LANG CLASS
lang_class = {0:[],1:[],2:[],3:[],4:[],5:[],6:[]}#6 is code mix -- may be we can add this to respective 2nd lang grp
class_lang = dd(list)
f=open("dataset/lang_class_mapping.csv");L=f.readlines();f.close()
#lang-name, code, class
for ele in L:
    ele = ele.strip("\n").strip().split("\t")
    if len(ele)>2:
        lang_class[int(ele[2].strip())].append((ele[0].strip(),ele[1].strip()))
        class_lang[ele[0].strip()].append(ele[2].strip())
    else:
        print(ele)
       
## NLP TRACK 
track_dataset = dd(list)
dataset_track = dd(list)
f=open("dataset/track_dataset_mapping.csv");L=f.readlines();f.close()
for ele in L[1:]:
    ele = ele.strip("\n").split("\t")
    if ele[1]:
        track_dataset[ele[0]] = [e.strip() for e in ele[1].split(",")]
        for data in ele[1].split(","):
            dataset_track[data.strip()] = ele[0]
    else:
        print(ele[0]," has no dataset")

## DS to store results

all_data = {0:dd(),1:dd(),2:dd(),3:dd(),4:dd(),5:dd(),6:dd()}
for class_id in all_data.keys():
    for lang in lang_class[class_id]:
        lang=lang[0]
        all_data[class_id][lang]=dd()
        for track in track_dataset:
            all_data[class_id][lang][track]=dd()
            for dataset in track_dataset[track]:
                all_data[class_id][lang][track][dataset] = 0.0
        print(class_id, lang, all_data[class_id][lang])    


def get_percent_change(lang_score, eng_score):
    # if lang_score > eng_score:
    #     cprint("Invalid assumption", "red")
    #     print(lang_score, eng_score, 100*np.double(eng_score - lang_score)/np.double(eng_score)) 
    percent_change = np.double(eng_score - lang_score)/np.double(eng_score)
    return percent_change*100

['Assamese', 'as']
['Czech', 'cs']
['Danish', 'da']
['Kashmiri', 'ka']
['Norwegian', 'no']
['Swedish', 'sv']
Computational Social Science and Cultural Analytics  has no dataset
Discourse and Pragmatics  has no dataset
Information Retrieval and Text Mining  has no dataset
Interpretability and Analysis of Models for NLP  has no dataset
Linguistic Theories, Cognitive Modeling, and Psycholinguistics  has no dataset
Machine Learning for NLP  has no dataset
Phonology, Morphology, and Word Segmentation  has no dataset
Resources and Evaluation  has no dataset
Semantics: Lexical  has no dataset
Speech and Multimodality  has no dataset
0 Bemba defaultdict(None, {'Dialogue and Interactive Systems': defaultdict(None, {'X-RiSAWOZ': 0.0}), 'Ethics and NLP': defaultdict(None, {'Jigsaw': 0.0, 'WinoMT': 0.0}), 'Generation': defaultdict(None, {'PAN-X': 0.0}), 'Language Grounding to Vision, Robotics and Beyond': defaultdict(None, {'MARVL': 0.0, 'XM3600': 0.0}), 'Machine Translation and Multilinguality': 

In [ ]:
'''
Code to read model data into the dict
'''
def get_model_scores(model_to_eval, all_data):
    f=open("dataset/genmodel_score.csv", encoding='windows-1254');L=f.readlines();f.close()
    L_header = L[0].split(",")
    ####
    model_index_dict = {'GPT-3.5-Turbo Monolingual':10, 'GPT-3.5-Turbo Translate-Test':11, 'GPT-3.5-Turbo Zero-Shot Cross Lingual':12, 
               'GPT-4 Monolingual':13, 
                'GPT-4 Translate-Test':14, 
                'GPT-4 Zero-Shot Cross Lingual':15, 
                'TULRv6':16, 
                'BLOOMZ':17, 
                'XLM-R':18, 
                'mBERT':19, 
                'MuRIL':20, 
                'XGLM':21, 
                'mT5':22, 
                'PaLM2 Zero-Shot':23, 
                'PaLM2 Monolingual':24, 
                'Llama2-7B Monolingual':25, 
                'Llama2-13B Monolingual':26, 
                'Llama2-70B Monolingual':27, 
                'LLaVa-1.5-13B Monolingual':28, 
                'LLaVa-1.5-13B Translate-Test':29}
    #####

    model_index = model_index_dict[model_to_eval]
    for ele in L[1:]:
        i=0
        ele = ele.split(",")
        class_id = class_lang[ele[2].strip()]
        lang = ele[2].strip()
        track = dataset_track[ele[0]].strip()
        dataset = ele[0].strip()
        #print(ele[model_index],class_id,lang,track,dataset,class_lang)
        if lang in ['Assamese','Czech','Danish', 'Kashmiri','Norwegian', 'Swedish']:
            continue
        all_data[int(class_id[0])][lang][track][dataset] = ele[model_index]
        
    return all_data

In [ ]:
def normalize_data(all_data):
    #scale values (scores on various langs) of all datasets to a scale of 100
    dataset_min_max = dd(list)
    for class_id in all_data.keys():
        for lang in all_data[class_id].keys():
            for track in all_data[class_id][lang].keys():
                if all_data[class_id][lang][track].keys():
                    for dataset in all_data[class_id][lang][track].keys():
                        val = all_data[class_id][lang][track][dataset]
                        if val!='NA' and val!='' and np.double(val) > 0.0:
                            dataset_min_max[dataset].append(np.double(all_data[class_id][lang][track][dataset]))
                        
    print(dataset_min_max)
    for dataset in dataset_min_max:
        dataset_min_max[dataset] = (np.double(np.min(dataset_min_max[dataset])), np.double(np.max(dataset_min_max[dataset])))

    print(dataset_min_max)

    #Update scores
    for class_id in all_data.keys():
        for lang in all_data[class_id].keys():
            for track in all_data[class_id][lang].keys():
                if all_data[class_id][lang][track].keys():
                    for dataset in all_data[class_id][lang][track].keys():
                        if  dataset in dataset_min_max.keys():
                            val = all_data[class_id][lang][track][dataset]
                            #print(val,dataset,dataset_min_max[dataset])
                            min_val, max_val = dataset_min_max[dataset]
                            if val!='NA' and val!='' and np.double(val) > 0.0:
                                val = np.double(val)
                                try:
                                    all_data[class_id][lang][track][dataset] = (val - min_val)/(max_val-min_val)
                                except:
                                    cprint(class_id,lang,track,dataset,val, min_val, max_val)
                        #print(class_id,lang,track,dataset,all_data[class_id][lang][track][dataset])


    return all_data

In [ ]:
'''
Code to get model lang scores - change wrt Eng scores ##Assumption Eng is best
O/P: standardized_all_data:: change wrt english on the same task. 
Ignore (do not include) if the score on the lang is not available or if the score for english on the task is not available
'''
def get_score_change_wrt_english(all_data, track_dataset):
    '''
    Trackwise English language scores for normalization
    '''
    english_norm_scores = dd() #Score of english language wrt to the the track -- taking mean...we can also take max scores for english

    for track in track_dataset:
        #cprint(track,"red")
        scores=[]
        for dataset in all_data[5]['English'][track].keys():
            #cprint(dataset,"blue")
            #print(all_data[5]['English'][track][dataset])
            if (all_data[5]['English'][track][dataset]) != "NA" and (all_data[5]['English'][track][dataset]) != "":
                scores.append(np.double(all_data[5]['English'][track][dataset]))
        if scores:
            if np.sum(scores) > 0.0:
                english_norm_scores[track] = np.mean([e for e in scores if e>0.0])
                #print("English score", english_norm_scores[track])
                
    #print(english_norm_scores)

    '''
    Use: english_norm_scores
    '''

    standardized_all_data = dd()
    standardized_all_data_missed = dd()

    for class_id in all_data.keys():
        standardized_all_data[class_id]=dd()
        standardized_all_data_missed[class_id]=dd()
        for lang in all_data[class_id].keys():
            if lang != "English":
                standardized_all_data[class_id][lang]=dd()
                standardized_all_data_missed[class_id][lang]=dd()
                for track in english_norm_scores.keys():
                    standardized_all_data[class_id][lang][track]=dd()
                    standardized_all_data_missed[class_id][lang][track]=dd()
                    for dataset in all_data[class_id][lang][track].keys():
                        if (all_data[class_id][lang][track][dataset] !='NA') and (all_data[class_id][lang][track][dataset]!='') and np.double(all_data[class_id][lang][track][dataset]) > 0.0:
                            standardized_all_data[class_id][lang][track][dataset] = get_percent_change(np.double(all_data[class_id][lang][track][dataset]), np.double(english_norm_scores[track]))
                        else:
                            standardized_all_data_missed[class_id][lang][track][dataset] = all_data[class_id][lang][track][dataset]
                            #missing_data.append(class_id,lang,track,dataset, all_data[class_id][lang][track][dataset])
                
    return standardized_all_data, standardized_all_data_missed

In [ ]:
def get_model_scores(standardized_all_data):
    class_wise_readiness_score = dd(list)
    track_wise_readiness_score = dd(list)
    for class_id in standardized_all_data.keys():
        for lang in standardized_all_data[class_id].keys():
            for track in standardized_all_data[class_id][lang].keys():
                if standardized_all_data[class_id][lang][track].keys():
                    for dataset in standardized_all_data[class_id][lang][track].keys():
                        #print(class_id, lang, track, dataset, standardized_all_data[class_id][lang][track][dataset])
                        class_wise_readiness_score[class_id].append(np.double(standardized_all_data[class_id][lang][track][dataset]))
                        track_wise_readiness_score[track].append(np.double(standardized_all_data[class_id][lang][track][dataset]))

    cprint(model_to_eval,"red")
    # print(class_wise_readiness_score)
    # for ele in class_wise_readiness_score.keys():
    #     print(ele, np.mean(class_wise_readiness_score[ele]))
        
    # print(track_wise_readiness_score)
    # for ele in track_wise_readiness_score.keys():
    #     print(ele, np.mean(track_wise_readiness_score[ele]))
        
    print("Overall Score");x=[]
    for ele in class_wise_readiness_score.keys():
        x.append(np.mean(np.abs(class_wise_readiness_score[ele])))
    print(np.mean(x))

    return class_wise_readiness_score, track_wise_readiness_score

In [5]:
####
for model_to_eval in model_index_dict.keys():
    all_data = get_model_scores(model_to_eval, all_data)
    all_data = normalize_data(all_data)
    standardized_all_data, standardized_all_data_missed = get_score_change_wrt_english(all_data, track_dataset)    
    ##
    class_wise_readiness_score, track_wise_readiness_score = get_model_scores(standardized_all_data)
                        


GPT-3.5-Turbo Monolingual
Overall Score
33.60142738276893
GPT-3.5-Turbo Translate-Test
Overall Score
16.922683682721832
GPT-3.5-Turbo Zero-Shot Cross Lingual
Overall Score
32.10036327989804
GPT-4 Monolingual
Overall Score
29.664362403576256
GPT-4 Translate-Test
Overall Score
6.461788819410208
GPT-4 Zero-Shot Cross Lingual
Overall Score
25.444646366772
TULRv6
Overall Score
4.449337427810926
BLOOMZ
Overall Score
33.30333032897285
XLM-R
Overall Score
22.794687581394545
mBERT
Overall Score
28.90385112058877
MuRIL
Overall Score
nan
XGLM
Overall Score
16.540168635900717
mT5
Overall Score
21.383861342694775
PaLM2 Zero-Shot
Overall Score
nan
PaLM2 Monolingual
Overall Score
53.69098169850285
Llama2-7B Monolingual
Overall Score
62.03804635495021
Llama2-13B Monolingual
Overall Score
68.71236020450606
Llama2-70B Monolingual
Overall Score
70.22350242085528
LLaVa-1.5-13B Monolingual
Overall Score
65.1876690288628
LLaVa-1.5-13B Translate-Test
Overall Score
nan


/home/t-jainprachi/project/code/MultilingualBlanketEval/megaenv/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/t-jainprachi/project/code/MultilingualBlanketEval/megaenv/lib/python3.8/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
